In [ ]:
from langgraph.graph import START,END,StateGraph
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import display,Image
from langchain_core.messages import HumanMessage,SystemMessage,RemoveMessage
from langgraph.graph import MessagesState
from langchain_core.messages import AnyMessage
from langgraph.graph import add_messages
from typing import Annotated
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",api_key=''  )


def multiply(a:int, b:int)->int:
    """Multiply a and b

    Args:
    a : First number
    b : Second number

    """
    return a * b

def divide(a:int, b:int)->float:
    """Divide a and b

    Args:
    a : First number
    b : Second number

    """
    return a / b

def add(a:int, b:int)->int:
    """Add a and b

    Args:
    a : First number
    b : Second number

    """
    return a + b

def subtract(a:int, b:int)->int:
    """Subtract a and b

    Args:
    a : First number
    b : Second number

    """
    return a - b
tools = [add, subtract, divide,multiply]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langgraph.graph import MessagesState,StateGraph,START
from langgraph.prebuilt import ToolNode,tools_condition
from langchain_core.messages import SystemMessage,HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import display,Image
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import AIMessage

class CustomState(TypedDict):
  messages: Annotated[list[AnyMessage], add_messages]


sys_message = SystemMessage(content='You are a helpful assistant tasked with performing arthemtic on a set of inputs')
def assistant_node(state:CustomState,config:RunnableConfig):
    return {'messages':[llm_with_tools.invoke([sys_message] + state['messages'])]}

def verifier_node(state:CustomState):

  current_state = compiling_graph.get_state(config1)
  existingMessages = current_state.values["messages"]
  if isinstance(existingMessages[-1],HumanMessage):
    lastMessageId = existingMessages[-1].id
    newMessage = HumanMessage(content='multiply 7 with 100',id = lastMessageId)
    finalMessages = add_messages(existingMessages,newMessage)
    compiling_graph.update_state(config1,{"messages":finalMessages})
    pass

building_graph = StateGraph(CustomState)
building_graph.add_node('verifier',verifier_node)
building_graph.add_node('assistant',assistant_node)
building_graph.add_node('tools',ToolNode(tools))

building_graph.add_edge(START,'verifier')
building_graph.add_edge('verifier','assistant')
building_graph.add_conditional_edges('assistant',tools_condition)
building_graph.add_edge('tools','assistant')

memory = MemorySaver()

compiling_graph = building_graph.compile(checkpointer=memory, interrupt_before=["verifier"] )

display(Image(compiling_graph.get_graph().draw_mermaid_png()))


In [ ]:
config1 = {'configurable':{'thread_id':'1',"user_id":123}}
input = {'messages':[HumanMessage(content='multiply 7 with 10')]}
x = compiling_graph.invoke(input,config1)
for message in x['messages']:
   message.pretty_print()


In [ ]:
from langchain_core.messages import AIMessage
current_state = compiling_graph.get_state(config1)
existingMessages = current_state.values["messages"]  #f616fd5b-712c-4e35-939a-bb9a514893fd
if isinstance(existingMessages[-1],HumanMessage):
  lastMessageId = existingMessages[-1].id
  newMessage = HumanMessage(content='multiply 7 with 100',id = lastMessageId)
  finalMessages = add_messages(existingMessages,newMessage)
  compiling_graph.update_state(config1,{"messages":finalMessages})


for message in compiling_graph.get_state(config1).values["messages"]:
    message.pretty_print()

In [ ]:
x=compiling_graph.invoke(None,config1)
for message in x['messages']:
    message.pretty_print()